In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import cv2
import random
import os

# Define the input video path and output directory
video_path = '/content/drive/MyDrive/Intern_assignment/videoplayback.mp4'
output_dir = '/content/drive/MyDrive/Intern_assignment/frames'
num_frames_to_capture = 8

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print(f"Error: Could not open video at {video_path}")
else:
    # Get total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        print("Error: Video has no frames.")
    else:
        # Generate random frame indices
        # Ensure we don't try to capture more frames than available
        if total_frames < num_frames_to_capture:
            print(f"Warning: Video has only {total_frames} frames. Capturing all available frames.")
            random_frame_indices = list(range(total_frames))
        else:
            random_frame_indices = sorted(random.sample(range(total_frames), num_frames_to_capture))

        print(f"Attempting to capture frames at indices: {random_frame_indices}")

        # Capture and save frames
        for i, frame_idx in enumerate(random_frame_indices):
            # Set the frame position
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)

            # Read the frame
            ret, frame = cap.read()

            if ret:
                # Define the filename for the captured frame
                frame_filename = os.path.join(output_dir, f"frame_{i+1}_{frame_idx}.jpg")

                # Save the frame
                cv2.imwrite(frame_filename, frame)
                print(f"Saved frame {frame_idx} as {frame_filename}")
            else:
                print(f"Warning: Could not read frame at index {frame_idx}.")

    # Release the video capture object
    cap.release()
    print("Video processing complete.")

# It's good practice to destroy any OpenCV windows, though often not visible in Colab
cv2.destroyAllWindows()

Attempting to capture frames at indices: [47, 103, 374, 619, 640, 654, 686, 750]
Saved frame 47 as /content/drive/MyDrive/Intern_assignment/frames/frame_1_47.jpg
Saved frame 103 as /content/drive/MyDrive/Intern_assignment/frames/frame_2_103.jpg
Saved frame 374 as /content/drive/MyDrive/Intern_assignment/frames/frame_3_374.jpg
Saved frame 619 as /content/drive/MyDrive/Intern_assignment/frames/frame_4_619.jpg
Saved frame 640 as /content/drive/MyDrive/Intern_assignment/frames/frame_5_640.jpg
Saved frame 654 as /content/drive/MyDrive/Intern_assignment/frames/frame_6_654.jpg
Saved frame 686 as /content/drive/MyDrive/Intern_assignment/frames/frame_7_686.jpg
Saved frame 750 as /content/drive/MyDrive/Intern_assignment/frames/frame_8_750.jpg
Video processing complete.


In [13]:
# Fix numpy binary-compatibility and reinstall ultralytics (run in Colab)
import os
import sys
from IPython.display import clear_output

# 1) Uninstall problematic packages (suppress noisy output)
print("Uninstalling ultralytics and numpy...")
!pip uninstall -y ultralytics numpy >/dev/null 2>&1 || true

# 2) Install a compatible numpy then ultralytics
print("Installing numpy==1.25.2 and ultralytics==8.2.103 ...")
!pip install --no-cache-dir numpy==1.25.2 --quiet
!pip install --no-cache-dir ultralytics==8.2.103 --quiet

# 3) Verify imports and model load
print("\nVerifying imports...")
import importlib, traceback
ok = True
try:
    import numpy as np
    print("numpy version:", np.__version__)
except Exception as e:
    ok = False
    print("numpy import FAILED")
    traceback.print_exc()

try:
    from ultralytics import YOLO
    print("ultralytics import: OK")
except Exception as e:
    ok = False
    print("ultralytics import FAILED")
    traceback.print_exc()

# 4) Try to load the model file if present to ensure no C-extension errors
model_file = "yolov8n-pose.pt"
if ok and os.path.exists(model_file):
    try:
        print(f"Attempting to load model file {model_file} ...")
        _ = YOLO(model_file)
        print("Model loaded successfully.")
    except Exception as e:
        ok = False
        print("Model failed to load:")
        traceback.print_exc()
else:
    print(f"Model file {model_file} not found locally — if not downloaded yet that's fine (Ultralytics will download when first used).")

# 5) If imports still failed, restart the runtime to ensure new binaries are picked up.
if not ok:
    print("\nInstallation may require a runtime restart. Restarting runtime now...")
    # Force a runtime restart (Colab will disconnect; re-run cells after reconnect).
    os.kill(os.getpid(), 9)
else:
    print("\nAll checks passed. You can re-run the YOLO inference cell now.")


Uninstalling ultralytics and numpy...
Installing numpy==1.25.2 and ultralytics==8.2.103 ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 269.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 301.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.1/875.1 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 172.3 MB/s eta 0:00:00
ERROR: pip's dependency re

**POSE ESTIMATION**

In [3]:
# Re-run YOLOv8-Pose inference: save annotated images + keypoints JSON (robust version)
from ultralytics import YOLO
from pathlib import Path
import os, json, cv2

FRAMES_DIR = "/content/drive/MyDrive/Intern_assignment/frames"
OUTPUT_DIR = "/content/drive/MyDrive/Intern_assignment/yolo_pose_outputs"
ANNOTATED_DIR = os.path.join(OUTPUT_DIR, "annotated")
KEYPOINTS_DIR = os.path.join(OUTPUT_DIR, "keypoints")
os.makedirs(ANNOTATED_DIR, exist_ok=True)
os.makedirs(KEYPOINTS_DIR, exist_ok=True)

# Load model (uses local copy if present; otherwise ultralytics downloads it)
model = YOLO("yolov8n-pose.pt")

image_paths = sorted(list(Path(FRAMES_DIR).glob("*.*")))
if len(image_paths) == 0:
    raise FileNotFoundError(f"No images found in {FRAMES_DIR}. Verify the path and files.")

print(f"Running pose inference on {len(image_paths)} images...")

for img_path in image_paths:
    img_name = img_path.name
    # Run inference (single image)
    res = model(str(img_path), imgsz=640)[0]  # take first Result

    # Annotated image: use res.plot() which returns an RGB numpy image
    try:
        annotated_rgb = res.plot()
        # Convert RGB -> BGR for cv2.imwrite
        annotated_bgr = cv2.cvtColor(annotated_rgb, cv2.COLOR_RGB2BGR)
        annotated_path = os.path.join(ANNOTATED_DIR, f"annotated_{img_name}")
        cv2.imwrite(annotated_path, annotated_bgr)
    except Exception as e:
        # Fallback: just copy original if plotting fails
        print(f"Warning: failed to create annotated image for {img_name}: {e}")
        annotated_path = os.path.join(ANNOTATED_DIR, f"annotated_{img_name}")
        orig = cv2.imread(str(img_path))
        cv2.imwrite(annotated_path, orig)

    # Extract keypoints
    kp_entries = []
    # results.keypoints may contain multiple persons; handle first person if present
    try:
        if hasattr(res, "keypoints") and len(res.keypoints) > 0:
            # res.keypoints.xy is a list of arrays (per detected instance)
            for inst_idx, inst_kp in enumerate(res.keypoints.xy):
                # inst_kp: Nx2 array of [x,y] for each keypoint (or nested structure)
                # Convert to python lists
                kp_xy = inst_kp.tolist()
                kp_entries.append({"instance_index": int(inst_idx), "keypoints_xy": kp_xy})
        else:
            kp_entries = []
    except Exception as e:
        print(f"Warning extracting keypoints for {img_name}: {e}")
        kp_entries = []

    out_json = {
        "image": img_name,
        "num_instances": len(kp_entries),
        "instances": kp_entries,
        # include model.names if available for keypoint label mapping (index -> label)
        "keypoint_labels": getattr(model, "names", None)
    }

    json_path = os.path.join(KEYPOINTS_DIR, f"{Path(img_name).stem}.json")
    with open(json_path, "w") as f:
        json.dump(out_json, f, indent=2)

    print(f"Processed {img_name}: annotated -> {annotated_path}, keypoints -> {json_path}")

print("\nInference complete.")
print("Annotated images saved to:", ANNOTATED_DIR)
print("Keypoints JSONs saved to:", KEYPOINTS_DIR)


Running pose inference on 8 images...

image 1/1 /content/drive/MyDrive/Intern_assignment/frames/frame_1_47.jpg: 384x640 2 persons, 14.6ms
Speed: 1.8ms preprocess, 14.6ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
Processed frame_1_47.jpg: annotated -> /content/drive/MyDrive/Intern_assignment/yolo_pose_outputs/annotated/annotated_frame_1_47.jpg, keypoints -> /content/drive/MyDrive/Intern_assignment/yolo_pose_outputs/keypoints/frame_1_47.json

image 1/1 /content/drive/MyDrive/Intern_assignment/frames/frame_2_103.jpg: 384x640 2 persons, 12.3ms
Speed: 1.8ms preprocess, 12.3ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
Processed frame_2_103.jpg: annotated -> /content/drive/MyDrive/Intern_assignment/yolo_pose_outputs/annotated/annotated_frame_2_103.jpg, keypoints -> /content/drive/MyDrive/Intern_assignment/yolo_pose_outputs/keypoints/frame_2_103.json

image 1/1 /content/drive/MyDrive/Intern_assignment/frames/frame_3_374.jpg: 384x640 2 persons, 11.

**ANGLE ESTIMATION**

In [4]:
# CODE BLOCK 2 — Compute shoulder–elbow–wrist (arm) angles using YOLOv8-Pose keypoints,
# save numeric results (CSV + summary TXT), and write annotated images with angles overlaid.
#
# Instructions:
# 1) Run this cell in Colab after you have run the YOLOv8 inference step that created
#    JSON keypoints in: /content/drive/MyDrive/Intern_assignment/yolo_pose_outputs/keypoints
# 2) This cell reads those JSONs, computes left & right elbow angles (shoulder->elbow->wrist),
#    saves results to a CSV, saves annotated images with angle text, and writes a short summary TXT.
# 3) If multiple person instances are present in a frame, this code uses the first detected instance.
#
# Output files (all saved under OUTPUT_DIR):
#   - angles_per_frame.csv         : frame filename, left_angle_deg, right_angle_deg
#   - annotated_angles_<frame>     : copy of annotated image with angle text overlays
#   - arm_angle_summary.txt        : short model-choice justification, calculation notes, and geometric interpretation
#
# Notes on mapping: the code will try to use 'keypoint_labels' found in the JSON to locate indices.
# If unavailable, it falls back to a default COCO-style index mapping:
#   left_shoulder=5, right_shoulder=6, left_elbow=7, right_elbow=8, left_wrist=9, right_wrist=10
# Adjust mapping manually in the MAPPING dictionary if your keypoint ordering differs.

import os
import json
import math
import csv
from pathlib import Path
import cv2
import numpy as np

# --- Paths (adjust if necessary) ---
KEYPOINTS_DIR = "/content/drive/MyDrive/Intern_assignment/yolo_pose_outputs/keypoints"
ANNOTATED_IN_DIR = "/content/drive/MyDrive/Intern_assignment/yolo_pose_outputs/annotated"
OUTPUT_DIR = "/content/drive/MyDrive/Intern_assignment/arm_angle_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)
ANNOTATED_OUT = os.path.join(OUTPUT_DIR, "annotated_with_angles")
os.makedirs(ANNOTATED_OUT, exist_ok=True)

CSV_PATH = os.path.join(OUTPUT_DIR, "angles_per_frame.csv")
SUMMARY_PATH = os.path.join(OUTPUT_DIR, "arm_angle_summary.txt")

# --- Helper math functions ---
def angle_between_points(a, b, c):
    """
    Compute angle at point b formed by points a-b-c (i.e., shoulder->elbow->wrist).
    Returns angle in degrees [0, 180]. a,b,c are (x,y) tuples or lists.
    If any point is None, returns None.
    """
    if a is None or b is None or c is None:
        return None
    ax, ay = a
    bx, by = b
    cx, cy = c
    # Vectors BA and BC
    v1 = (ax - bx, ay - by)
    v2 = (cx - bx, cy - by)
    # Compute dot and norms
    dot = v1[0]*v2[0] + v1[1]*v2[1]
    n1 = math.hypot(v1[0], v1[1])
    n2 = math.hypot(v2[0], v2[1])
    if n1 == 0 or n2 == 0:
        return None
    cos_theta = max(-1.0, min(1.0, dot / (n1 * n2)))
    theta = math.degrees(math.acos(cos_theta))
    return theta

# --- Attempt to read all JSON keypoint files ---
kp_files = sorted(list(Path(KEYPOINTS_DIR).glob("*.json")))
if len(kp_files) == 0:
    raise FileNotFoundError(f"No keypoint JSON files found in {KEYPOINTS_DIR}. Run YOLOv8-Pose inference first.")

results_rows = []
frames_used = []

# Default fallback mapping (COCO-like). Modify if your JSON reveals a different order.
FALLBACK_MAPPING = {
    "left_shoulder": 5,
    "right_shoulder": 6,
    "left_elbow": 7,
    "right_elbow": 8,
    "left_wrist": 9,
    "right_wrist": 10
}

# Process each keypoint JSON
for kp_file in kp_files:
    with open(kp_file, "r") as f:
        data = json.load(f)
    image_name = data.get("image", Path(kp_file).stem)
    frames_used.append(image_name)

    # Attempt to pick the first instance (if multiple persons detected)
    instances = data.get("instances", [])
    if len(instances) == 0:
        # No keypoints - record Nones
        left_angle = None
        right_angle = None
        results_rows.append((image_name, left_angle, right_angle))
        continue

    inst = instances[0]  # first detected person
    kp_xy = inst.get("keypoints_xy", None)
    if kp_xy is None:
        # legacy structure support: maybe data has keypoints_xy at root
        kp_xy = data.get("keypoints_xy", None)

    if kp_xy is None:
        left_angle = None
        right_angle = None
        results_rows.append((image_name, left_angle, right_angle))
        continue

    # Determine mapping from labels if available
    mapping = {}
    labels = data.get("keypoint_labels", None)
    if labels and isinstance(labels, (list, dict)):
        # labels can be a list (index -> label) or dict; normalize to list if possible
        if isinstance(labels, dict):
            # dict mapping; try to invert if needed
            label_list = [labels.get(str(i)) for i in range(len(kp_xy))] if len(labels) >= len(kp_xy) else None
        else:
            label_list = labels
        # Search for common names
        if label_list:
            for i, lab in enumerate(label_list):
                lname = str(lab).lower()
                if "left" in lname and ("shoulder" in lname or "sho" in lname):
                    mapping["left_shoulder"] = i
                if "right" in lname and ("shoulder" in lname or "sho" in lname):
                    mapping["right_shoulder"] = i
                if "left" in lname and ("elbow" in lname):
                    mapping["left_elbow"] = i
                if "right" in lname and ("elbow" in lname):
                    mapping["right_elbow"] = i
                if "left" in lname and ("wrist" in lname):
                    mapping["left_wrist"] = i
                if "right" in lname and ("wrist" in lname):
                    mapping["right_wrist"] = i

    # Fill missing mapping fields with fallback indices
    for k, fallback_idx in FALLBACK_MAPPING.items():
        if k not in mapping:
            mapping[k] = fallback_idx

    # Safely index into kp_xy (guard against out-of-range)
    def safe_get(idx):
        try:
            pt = kp_xy[idx]
            if pt is None:
                return None
            # Some outputs may contain [x, y, conf]; handle that
            if len(pt) >= 2:
                return (float(pt[0]), float(pt[1]))
        except Exception:
            return None
        return None

    left_sh = safe_get(mapping["left_shoulder"])
    left_el = safe_get(mapping["left_elbow"])
    left_wr = safe_get(mapping["left_wrist"])
    right_sh = safe_get(mapping["right_shoulder"])
    right_el = safe_get(mapping["right_elbow"])
    right_wr = safe_get(mapping["right_wrist"])

    left_angle = angle_between_points(left_sh, left_el, left_wr)
    right_angle = angle_between_points(right_sh, right_el, right_wr)

    results_rows.append((image_name, left_angle, right_angle))

    # Annotate existing annotated image (if available) with angle text and save to OUTPUT folder
    annotated_in_path = os.path.join(ANNOTATED_IN_DIR, image_name)
    if not os.path.exists(annotated_in_path):
        # try common prefix "annotated_"
        alt_path = os.path.join(ANNOTATED_IN_DIR, f"annotated_{image_name}")
        if os.path.exists(alt_path):
            annotated_in_path = alt_path

    if os.path.exists(annotated_in_path):
        im = cv2.imread(annotated_in_path)
        if im is None:
            # skip annotation if image can't be read
            continue
        # Overlay left/right angle near elbows (if available)
        font = cv2.FONT_HERSHEY_SIMPLEX
        if left_el is not None and left_angle is not None:
            lx, ly = int(left_el[0]), int(left_el[1])
            text = f"L:{left_angle:.1f}deg"
            cv2.putText(im, text, (lx+5, ly-10), font, 0.6, (0,255,0), 2, cv2.LINE_AA)
        if right_el is not None and right_angle is not None:
            rx, ry = int(right_el[0]), int(right_el[1])
            text = f"R:{right_angle:.1f}deg"
            cv2.putText(im, text, (rx+5, ry-10), font, 0.6, (0,255,0), 2, cv2.LINE_AA)

        out_ann_path = os.path.join(ANNOTATED_OUT, f"angles_{image_name}")
        cv2.imwrite(out_ann_path, im)

# --- Write CSV of results ---
with open(CSV_PATH, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["frame", "left_arm_angle_deg", "right_arm_angle_deg"])
    for row in results_rows:
        # format None as empty string
        writer.writerow([row[0],
                         "" if row[1] is None else f"{row[1]:.3f}",
                         "" if row[2] is None else f"{row[2]:.3f}"])

# --- Prepare short summary text (model-choice justification + calculation + interpretation) ---
# Model-choice justification (1-2 sentences)
model_choice_justification = (
    "Model choice: YOLOv8-Pose (ultralytics). Chosen because it installs and runs reliably in this Python/Colab "
    "environment, downloads a small pose model automatically, and provides direct keypoint outputs suitable for "
    "computing joint angles quickly."
)

# Calculation explanation (2-3 sentences)
calc_explanation = (
    "Calculation: the elbow angle is computed at the elbow landmark using the three 2D points "
    "(shoulder -> elbow -> wrist). The angle is the arccosine of the dot product between vectors (shoulder - elbow) "
    "and (wrist - elbow), converted to degrees."
)

# Simple geometric interpretation: compute deltas between first and last frames when possible
interpret_lines = []
# gather numeric arrays for left & right
left_vals = [r[1] for r in results_rows if r[1] is not None]
right_vals = [r[2] for r in results_rows if r[2] is not None]
if len(left_vals) >= 2:
    left_delta = left_vals[-1] - left_vals[0]
    interpret_lines.append(f"Left elbow angle changed by {left_delta:.1f}° between first and last measured frames.")
elif len(left_vals) == 1:
    interpret_lines.append("Left elbow angle measured in one frame only — no across-frame change computed.")
else:
    interpret_lines.append("Left elbow angle could not be measured in any frames.")

if len(right_vals) >= 2:
    right_delta = right_vals[-1] - right_vals[0]
    interpret_lines.append(f"Right elbow angle changed by {right_delta:.1f}° between first and last measured frames.")
elif len(right_vals) == 1:
    interpret_lines.append("Right elbow angle measured in one frame only — no across-frame change computed.")
else:
    interpret_lines.append("Right elbow angle could not be measured in any frames.")

# Detailed per-frame geometric interpret summary (2-3 lines)
per_frame_lines = []
for img, la, ra in results_rows:
    la_str = "NA" if la is None else f"{la:.1f}°"
    ra_str = "NA" if ra is None else f"{ra:.1f}°"
    per_frame_lines.append(f"{img}: Left={la_str}, Right={ra_str}")

with open(SUMMARY_PATH, "w") as sf:
    sf.write("=== Model choice justification ===\n")
    sf.write(model_choice_justification + "\n\n")
    sf.write("=== Calculation explanation ===\n")
    sf.write(calc_explanation + "\n\n")
    sf.write("=== Geometric interpretation (summary) ===\n")
    for line in interpret_lines:
        sf.write(line + "\n")
    sf.write("\n=== Per-frame angles ===\n")
    for line in per_frame_lines:
        sf.write(line + "\n")

# --- Print concise results to notebook output for quick inspection ---
print("Frames used (in order):")
for f in frames_used:
    print(" -", f)

print("\nComputed angles (Left, Right) in degrees:")
for img, la, ra in results_rows:
    la_s = "NA" if la is None else f"{la:.1f}"
    ra_s = "NA" if ra is None else f"{ra:.1f}"
    print(f"{img}: Left={la_s}°, Right={ra_s}°")

print(f"\nCSV saved to: {CSV_PATH}")
print(f"Annotated images with angles saved to: {ANNOTATED_OUT}")
print(f"Text summary (justification + calculation + interpretation) saved to: {SUMMARY_PATH}")

# End of CODE BLOCK 2


Frames used (in order):
 - frame_1_47.jpg
 - frame_2_103.jpg
 - frame_3_374.jpg
 - frame_4_619.jpg
 - frame_5_640.jpg
 - frame_6_654.jpg
 - frame_7_686.jpg
 - frame_8_750.jpg

Computed angles (Left, Right) in degrees:
frame_1_47.jpg: Left=155.2°, Right=156.6°
frame_2_103.jpg: Left=150.4°, Right=169.5°
frame_3_374.jpg: Left=7.3°, Right=73.7°
frame_4_619.jpg: Left=116.6°, Right=142.8°
frame_5_640.jpg: Left=103.8°, Right=131.2°
frame_6_654.jpg: Left=95.1°, Right=134.7°
frame_7_686.jpg: Left=64.3°, Right=102.4°
frame_8_750.jpg: Left=67.1°, Right=150.7°

CSV saved to: /content/drive/MyDrive/Intern_assignment/arm_angle_outputs/angles_per_frame.csv
Annotated images with angles saved to: /content/drive/MyDrive/Intern_assignment/arm_angle_outputs/annotated_with_angles
Text summary (justification + calculation + interpretation) saved to: /content/drive/MyDrive/Intern_assignment/arm_angle_outputs/arm_angle_summary.txt


**FRAME DESCRIPTION**

In [6]:
# Generate one-sentence pose estimates per frame and save to a text file in a separate folder.
# - Uses existing YOLOv8-Pose keypoints JSONs if present (faster).
# - Otherwise runs YOLOv8-pose on each image to obtain keypoints.
# - Produces one concise, exam-style sentence per frame describing pose (elbow angles, arms up/down, shoulder tilt).
# - Saves the text file to: /content/drive/MyDrive/Intern_assignment/pose_estimates/pose_descriptions.txt
#
# Run this in Colab after mounting Drive. Assumes ultralytics is installed and workable.
# If ultralytics import fails, the script will try only to use existing JSONs and will error if none exist.

import os
from pathlib import Path
import json
import math
import cv2
from ultralytics import YOLO

# --- User paths ---
FRAMES_DIR = Path("/content/drive/MyDrive/Intern_assignment/frames")
YOLO_KEYPOINTS_DIR = Path("/content/drive/MyDrive/Intern_assignment/yolo_pose_outputs/keypoints")
OUTPUT_TEXT_DIR = Path("/content/drive/MyDrive/Intern_assignment/pose_estimates")
OUTPUT_TEXT_DIR.mkdir(parents=True, exist_ok=True)
OUT_TEXT_PATH = OUTPUT_TEXT_DIR / "pose_descriptions.txt"

# --- Helper math functions ---
def angle_between(a, b, c):
    """Angle at b formed by a-b-c in degrees. Returns float or None."""
    if a is None or b is None or c is None:
        return None
    ax, ay = a; bx, by = b; cx, cy = c
    v1 = (ax - bx, ay - by)
    v2 = (cx - bx, cy - by)
    dot = v1[0]*v2[0] + v1[1]*v2[1]
    n1 = math.hypot(*v1)
    n2 = math.hypot(*v2)
    if n1 == 0 or n2 == 0:
        return None
    cosv = max(-1.0, min(1.0, dot / (n1*n2)))
    return math.degrees(math.acos(cosv))

def safe_get_point(kp_list, idx):
    """Return (x,y) or None if unavailable."""
    try:
        pt = kp_list[idx]
        if pt is None:
            return None
        if len(pt) >= 2:
            return (float(pt[0]), float(pt[1]))
    except Exception:
        return None
    return None

# --- Common label order used by ultralytics/yolov8-pose (fallback) ---
DEFAULT_LABELS = [
    "nose","left_eye","right_eye","left_ear","right_ear",
    "left_shoulder","right_shoulder","left_elbow","right_elbow",
    "left_wrist","right_wrist","left_hip","right_hip",
    "left_knee","right_knee","left_ankle","right_ankle"
]
label_to_idx_default = {lab: i for i, lab in enumerate(DEFAULT_LABELS)}

# --- Load existing keypoints JSONs (if available) ---
def load_keypoints_jsons(kp_dir):
    files = sorted(kp_dir.glob("*.json"))
    kp_map = {}
    for jf in files:
        try:
            with open(jf, "r") as f:
                data = json.load(f)
            fname = data.get("image", jf.stem)
            kp_map[fname] = data
        except Exception:
            continue
    return kp_map

existing_kp = {}
if YOLO_KEYPOINTS_DIR.exists():
    existing_kp = load_keypoints_jsons(YOLO_KEYPOINTS_DIR)

# --- Prepare list of frame file names (preserve order) ---
img_paths = sorted([p for p in FRAMES_DIR.glob("*") if p.suffix.lower() in [".jpg",".jpeg",".png",".bmp"]])
if not img_paths:
    raise FileNotFoundError(f"No images found in {FRAMES_DIR}")

# --- If no existing keypoints, prepare to run YOLOv8-pose ---
need_model = False
for p in img_paths:
    if p.name not in existing_kp:
        need_model = True
        break

model = None
if need_model:
    try:
        model = YOLO("yolov8n-pose.pt")  # will download model if missing
    except Exception as e:
        # If model load fails and there are no keypoints at all, we cannot continue.
        if not existing_kp:
            raise RuntimeError("YOLOv8 model could not be loaded and no existing keypoints found. Error: " + str(e))
        model = None

# --- Function to obtain keypoints list for a frame (first instance) ---
def get_keypoints_for_image(img_path, existing_map, model):
    """Returns (image_name, width, height, kp_xy_list, keypoint_labels)"""
    img_name = img_path.name
    # Priority: existing JSON
    if img_name in existing_map:
        data = existing_map[img_name]
        instances = data.get("instances", [])
        if instances:
            kpxy = instances[0].get("keypoints_xy", [])
        else:
            kpxy = []
        w = data.get("width", None)
        h = data.get("height", None)
        labels = data.get("keypoint_labels", None)
        return img_name, w, h, kpxy, labels
    # Else run model inference
    if model is None:
        return img_name, None, None, [], None
    res = model(str(img_path), imgsz=640)[0]
    # try to get image size
    img = cv2.imread(str(img_path))
    h, w = (img.shape[0], img.shape[1]) if img is not None else (None, None)
    kpxy = []
    labels = getattr(model, "names", None)
    try:
        if hasattr(res, "keypoints") and len(res.keypoints) > 0:
            # take first instance
            kpxy = res.keypoints.xy[0].tolist()
        else:
            kpxy = []
    except Exception:
        kpxy = []
    return img_name, w, h, kpxy, labels

# --- Compose one-sentence description for single frame ---
def describe_pose_single(img_name, w, h, kpxy, labels):
    """Return one concise sentence describing pose for this frame."""
    # Build index map
    idx_map = {}
    if labels:
        # labels may be a dict or list
        if isinstance(labels, dict):
            # try to build list by numeric keys
            try:
                label_list = [labels.get(str(i)) for i in range(len(kpxy))]
            except Exception:
                label_list = None
        else:
            label_list = labels
        if isinstance(label_list, list) and all(label_list):
            idx_map = {str(l).lower(): i for i, l in enumerate(label_list)}
    if not idx_map:
        idx_map = {k: v for k, v in label_to_idx_default.items()}

    def pt(name):
        ix = idx_map.get(name.lower())
        if ix is None:
            return None
        return safe_get_point(kpxy, ix)

    nose = pt("nose")
    l_sh = pt("left_shoulder"); r_sh = pt("right_shoulder")
    l_el = pt("left_elbow"); r_el = pt("right_elbow")
    l_wr = pt("left_wrist"); r_wr = pt("right_wrist")
    l_hip = pt("left_hip"); r_hip = pt("right_hip")

    # numeric features
    left_elbow_ang = angle_between(l_sh, l_el, l_wr)
    right_elbow_ang = angle_between(r_sh, r_el, r_wr)

    # wrists relative to shoulders (vertical)
    def wrist_vs_sh(wr, sh):
        if wr is None or sh is None or h is None:
            return None
        # Negative means wrist is above shoulder (arm raised)
        return float(wr[1]) - float(sh[1])

    l_wrs = wrist_vs_sh(l_wr, l_sh)
    r_wrs = wrist_vs_sh(r_wr, r_sh)

    # Shoulder tilt angle (left->right)
    def shoulder_tilt_deg(ls, rs):
        if ls is None or rs is None:
            return None
        dx = rs[0] - ls[0]
        dy = rs[1] - ls[1]
        return math.degrees(math.atan2(dy, dx))

    sh_tilt = shoulder_tilt_deg(l_sh, r_sh)

    # Compose description parts
    parts = []
    if left_elbow_ang is not None:
        parts.append(f"left elbow ~{left_elbow_ang:.0f}°")
    if right_elbow_ang is not None:
        parts.append(f"right elbow ~{right_elbow_ang:.0f}°")

    # arms up/down detection using image height scale
    arm_note = None
    if h:
        thr = 0.03 * h  # 3% image height
        left_pos = None
        right_pos = None
        if l_wrs is not None:
            left_pos = "raised" if l_wrs < -thr else ("lowered" if l_wrs > thr else "neutral")
        if r_wrs is not None:
            right_pos = "raised" if r_wrs < -thr else ("lowered" if r_wrs > thr else "neutral")
        if left_pos and right_pos:
            if left_pos == right_pos:
                arm_note = f"both arms {left_pos}" if left_pos != "neutral" else None
            else:
                # prefer naming which arm is raised/lowered
                notes = []
                if left_pos != "neutral":
                    notes.append(f"left arm {left_pos}")
                if right_pos != "neutral":
                    notes.append(f"right arm {right_pos}")
                arm_note = ", ".join(notes)
        elif left_pos:
            arm_note = f"left arm {left_pos}" if left_pos != "neutral" else None
        elif right_pos:
            arm_note = f"right arm {right_pos}" if right_pos != "neutral" else None
    if arm_note:
        parts.append(arm_note)

    if sh_tilt is not None:
        # small rounding, indicate tilt direction
        direction = "tilt clockwise" if sh_tilt > 6 else ("tilt counterclockwise" if sh_tilt < -6 else None)
        if direction:
            parts.append(f"shoulder {direction} {sh_tilt:.1f}°")

    # If no meaningful parts, say pose detected but incomplete
    if not parts:
        sentence = f"{img_name}: Pose detected but keypoint estimates incomplete."
    else:
        sentence = f"{img_name}: " + "; ".join(parts) + "."

    return sentence

# --- Main loop: build descriptions for all frames, save text file ---
lines = []
for img_path in img_paths:
    name, w, h, kpxy, labels = get_keypoints_for_image(img_path, existing_kp, model)
    # If kpxy is empty, attempt to run model again (robustness)
    if (not kpxy) and model is not None:
        try:
            name, w, h, kpxy, labels = get_keypoints_for_image(img_path, existing_kp, model)
        except Exception:
            pass
    desc = describe_pose_single(name, w, h, kpxy, labels)
    lines.append(desc)

# Save to text file
with open(OUT_TEXT_PATH, "w") as f:
    for line in lines:
        f.write(line + "\n")

# Also print brief confirmation + first few lines
print(f"Wrote {len(lines)} pose descriptions to: {OUT_TEXT_PATH}\n")
for l in lines[:min(8, len(lines))]:
    print(l)


Wrote 8 pose descriptions to: /content/drive/MyDrive/Intern_assignment/pose_estimates/pose_descriptions.txt

frame_1_47.jpg: left elbow ~155°; right elbow ~157°; shoulder tilt clockwise 175.4°.
frame_2_103.jpg: left elbow ~150°; right elbow ~170°; shoulder tilt clockwise 177.4°.
frame_3_374.jpg: left elbow ~7°; right elbow ~74°; shoulder tilt counterclockwise -151.7°.
frame_4_619.jpg: left elbow ~117°; right elbow ~143°; shoulder tilt clockwise 11.7°.
frame_5_640.jpg: left elbow ~104°; right elbow ~131°; shoulder tilt clockwise 13.5°.
frame_6_654.jpg: left elbow ~95°; right elbow ~135°; shoulder tilt clockwise 8.2°.
frame_7_686.jpg: left elbow ~64°; right elbow ~102°; shoulder tilt clockwise 11.8°.
frame_8_750.jpg: left elbow ~67°; right elbow ~151°.
